In [1]:
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

import time
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings("ignore")

In [2]:
base_path = 'D:/DataSet/Credit/'
train = pd.read_csv(base_path + 'featured/train_all_feature.csv')
test = pd.read_csv(base_path + 'featured/test_all_feature.csv')

one_hot = 0
if one_hot:
    train = pd.get_dummies(train)
    test = pd.get_dummies(test)
    
col_to_drop = [
    # 1
    'count_house_loan_ln',# 0.97232912500359225)
    'count_house_loan',# 0.65395611105534779)
    'edu_level_other',# 0.57350198625873861)
    'count_payment_state_E_ln',# 0.51044888053676218)
    'count_attention_ln',# 0.32350891467937332)
    'count_commercial_loan',# 0.0)
    'count_sixty_ovd_dw',# 0.0)
    'count_sixty_ovd_months',# 0.0)
    'ind_sixty_max_duration',# 0.0)
    'marry_status_other',# 0.0)
    'count_study_loan_ln',# 0.0)
    'count_housing_accumulation_ln',# 0.0)
    'count_commercial_housing_ln',# 0.0)
    'count_combination_ensure_ln',# 0.0)
    'ind_other_counts_lnd',# 0.0)
    'count_combination_lnd',# 0.0)
    'count_pledge_guarantee_lnd',# 0.0)
    'count_ensure_lnd',# 0.0)
    'count_other_guarantee_lnd',# 0.0)
    'count_combination_ensure_lnd',# 0.0)
    'count_farmer_joint_lnd',# 0.0)
    'count_pledge_guarantee_bail_lnd',# 0.0)
    # 10
    'ind_curr_overdue_cyc_lnd',# 9.1747017468829988)
    'ind_unact_counts_lnd',# 8.1829607049772104)
    'ind_other_counts',# 8.1190294047611253)
    'count_pledge_guarantee_bail_ln',# 4.9866824922417035)
    'count_sharedebt',# 4.25470982342895)
    'marry_status_unmarried',# 3.5268450817329375)
    'count_normal_ln',# 3.1544560288384238)
    'count_farmer_joint_ln',# 2.8250789765495536)
    'not_clear_account_count',# 2.438362051462843)
    'count_car_loan_ln',# 2.0244134983035815)
    'count_payment_state_E_lnd',# 1.9511014375106948)
    # 40
    'ind_normal_counts',# 38.981578298058366)
    'count_ensure_ln',# 32.686012781155711)
    'not_logout_pre_account_count',# 29.035934862113912)
    'count_spl',# 26.621401271906876)
    'not_logout_pre_finance_org_count',# 20.88833411697636)
    'count_debit_card_ovd_dw',# 20.552771641255653)
    'ind_clear_counts_lnd',# 18.63633007759838)
    'cat_query_reason_mal',# 18.012892273855371)
    'count_other_guarantee_ln',# 15.719418555370231)
    'count_pledge_guarantee_ln',# 14.603172531948651)
    'count_standard_loancard',# 14.257967916620288)
    'count_combination_ln',# 13.225866749150008)
    'marry_status_divorced',# 12.996673186745294)
    'flt_highest_sixty_oa_per_mon',# 12.496437876453976)
    'count_farmer_loan_ln',# 12.034259912867824)
    # 60
    'curr_overdue_cyc_days',# 58.533138767951407)
    'not_logout_pre_max_credit_limit_per_org',# 55.545770066417433)
    'not_logout_pre_finance_corp_count',# 54.909995520093744)
    'has_fund',# 53.524477466077244)
    'edu_level_bachelor',# 51.681921368379456)
    'cat_query_reason_sqe',# 50.507370364531575)
    'not_clear_finance_org_count',# 50.499752342403504)
    'count_payment_state_B_ln',# 49.675949679726415)
    'count_ovd',# 47.284280675138668)
    'marry_status_married',# 47.141063732728689)
    # 100
    'count_payment_state_D_ln',# 98.582591891500016)
    'not_logout_finance_org_count',# 97.63379220401383)
    'count_credit_loan_ln',# 96.057688729626022)
    'count_payment_state_D_lnd',# 94.52677681813546)
    'all_highest_oa_per_mon',# 93.758390716396264)
    'count_consumption loan_ln',# 92.34403542313882)
    'not_logout_pre_latest_6m_used_avg_amount',# 85.357040829001591)
    'balance',# 85.186190705260103)
    'cat_query_reason_la',# 83.737758346562558)
    'ind_clear_counts',# 82.467554392775014)
    'count_unknown_ln',# 73.400641666623201)
    'not_logout_pre_min_credit_limit_per_org',# 70.752886069309909)
    'not_logout_pre_credit_limit',# 68.394379619115668)
    'not_logout_account_count',# 67.926934887483739)
    'lnd_ovd_sum_amount',# 67.598474433716945)
    'flt_highest_debit_card_oa_per_mon',# 64.9057845815354)
    'count_loan_ovd_dw',# 64.132474960621153)
    'count_operating_loan_ln',# 63.127772355688421)
    'not_clear_finance_corp_count',# 62.877948676798582)
    # 180
    'not_logout_latest_6m_used_avg_amount',# 176.32317263727683)
    'ind_loan_max_duration',# 167.56724876084013)
    'count_credit_loan_lnd',# 166.96045957856194)
    'all_max_duration',# 153.57957111737943)
    'not_clear_balance',# 153.18548993022915)
    'flt_highest_loan_oa_per_mon',# 150.5264464131252)
    'flt_sum_amount',# 149.83162201460962)
    'used_highest_amount_lnd',# 136.85598322859255)
    'scheduled_payment_amount',# 128.02030046387119)
    'latest6_month_used_avg_amount_lnd',# 124.58135686300396)
    'ind_org_counts',# 120.5490321578975)
    'ind_debit_card_max_duration',# 118.53218655429413)
    'edu_level_junior',# 117.02178169627837)
    'lnd_ovd_sum_last_months',# 114.74345491704378)
    'used_credit_limit_amount_lnd',# 114.2672848868574)
    'ave_ovd_amount',# 114.06571798355444)
    'count_other_loan',# 111.82596886962884)
    'range_lnd_ovd',# 109.13363916690426)
    'not_logout_pre_used_credit_limit',# 105.64340446411771)
    'count_sum_ovd_dw',# 104.97563024241767)
    # flt_noise
    'not_logout_max_credit_limit_per_org',# 211.80030432198566)
    # 240
    #'not_logout_finance_corp_count',# 217.97120979038462)
]

train = train.drop(col_to_drop, axis=1)
test = test.drop(col_to_drop,axis=1)

In [3]:
features = train.columns.drop(['y','report_id'])    # 列名
X = train.drop(['y','report_id'],axis=1)            # 不包含 target 的数据集
y = train.y                                         # 目标数据
sub=test.report_id.to_frame()                       # 用于储存结果
sub['pred']=0                                       # 初始化为 0
sub_train=train.report_id.to_frame()
sub_train['y']=train.y                              # oof
sub_train['pred'] = 0

all_gini = []

#log_x = [0,0.2,0.4,0.6,0.8,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3]

#tree = np.ceil(np.power(10,log_x)).astype(np.int32)

#for n in tree:
rf = RandomForestClassifier(n_estimators = 1000,
                            min_samples_split=60, 
                            max_depth=10, 
                            #max_features=None,
                            max_features='auto',# None then max_features=n_features.
                            min_samples_leaf=20,
                            min_weight_fraction_leaf=0,
                            max_leaf_nodes=None,
                            min_impurity_decrease=0,
                            bootstrap=True,
                            oob_score=False, 
                            n_jobs=-1,
                            verbose=0,
                            class_weight={1:5, 0:1},# 'balanced',
                            criterion='entropy')

K = 5
GINI = []
for i in range(20):
    skf = StratifiedKFold(n_splits=K, random_state=int(time.time()), shuffle = True) 
    for i, (train_index, valid_index) in enumerate(skf.split(train, y)):
        #print(' lgb kfold: {}  of  {} : '.format(i+1, K))
        X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[valid_index,:].copy()
        y_train, y_valid = y.iloc[train_index].copy(), y.iloc[valid_index]
        X_test = test.drop(['report_id'],axis=1)
    
        rf.fit(X_train,y_train)
    
        pred_valid = rf.predict_proba(X_valid)[:,1]
        sub_train['pred'].iloc[valid_index] = pred_valid   
        print( "  Gini = ", eval_gini(y_valid, pred_valid) )
        sub['pred'] += rf.predict_proba(X_test)[:,1]/K
    
    print( "\nGini for full training set:" )
    print(eval_gini(y, sub_train.pred))
    GINI.append(eval_gini(y, sub_train.pred))

  Gini =  0.755236503704
  Gini =  0.695591348148
  Gini =  0.731659851852
  Gini =  0.726962725926
  Gini =  0.726722844444

Gini for full training set:
0.726689507556
  Gini =  0.758134992593
  Gini =  0.712941511111
  Gini =  0.720850962963
  Gini =  0.730142814815
  Gini =  0.719937896296

Gini for full training set:
0.727780446815
  Gini =  0.720703051852
  Gini =  0.732204088889
  Gini =  0.723780740741
  Gini =  0.703609837037
  Gini =  0.746731614815

Gini for full training set:
0.724870618074
  Gini =  0.734217007407
  Gini =  0.694366340741
  Gini =  0.739840474074
  Gini =  0.725314844444
  Gini =  0.744441837037

Gini for full training set:
0.727098538667
  Gini =  0.747634251852
  Gini =  0.72678162963
  Gini =  0.726038281481
  Gini =  0.723510518519
  Gini =  0.719804207407

Gini for full training set:
0.728416350815
  Gini =  0.724675792593
  Gini =  0.712124207407
  Gini =  0.707726696296
  Gini =  0.738201125926
  Gini =  0.750357333333

Gini for full training set:
0.

In [21]:
sub_train.to_csv(base_path+'result/RF_train.csv', index=False)
sub.to_csv(base_path+'result/RF_test.csv', index=False)

In [5]:
GINI

[0.7266895075555555,
 0.7277804468148148,
 0.72487061807407405,
 0.72709853866666663,
 0.72841635081481482,
 0.72629101985185185,
 0.72920498251851851,
 0.72892709925925925,
 0.7257895253333333,
 0.72771320414814822,
 0.72748963081481488,
 0.72599542518518523,
 0.72701498785185192,
 0.72737149155555558,
 0.72638913422222218,
 0.72785129244444446,
 0.7272233908148148,
 0.72860207407407407,
 0.72695794725925933,
 0.72880323318518525]